In [31]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [64]:
# Read CSV file

data=pd.read_csv('preprocessed_UK_Accidents_2009_updated.csv',index_col='accident_index')
data=data.drop('seasons_ranges',axis=1)
x = data.drop('accident_severity', axis=1) 
y = data['accident_severity']

In [65]:
# Scalling data
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [66]:
from sklearn.decomposition import PCA
priciple_component_analayzer = PCA(n_components=15) # based on domain knowledge from feature extraction notebook
x_pca = priciple_component_analayzer.fit_transform(x_scaled)

In [36]:
## Over sampling due to data imbalance 
oversampler = RandomOverSampler(random_state=10)
x_resampled, y_resampled = oversampler.fit_resample(x_pca, y)
x_resampled=pd.DataFrame(x_resampled)


In [37]:
## resampling effect
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
input_dim = X_train.shape[1]
y_train_new = np.subtract(y_train, 1)
y_test_new = np.subtract(y_test, 1)
y_val_new = np.subtract(y_val, 1)

# Convert the new class labels to a one-hot encoded array
y_train_onehot = to_categorical(y_train_new, num_classes=3)
y_test_onehot = to_categorical(y_test_new, num_classes=3)
y_val_onehot  =  to_categorical(y_val_new, num_classes=3)
# Define the model architecture
model = Sequential([
    Dense(64, activation='relu', input_shape=(input_dim,)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Train the model
model.fit(X_train, y_train_onehot, epochs=10, batch_size=32, validation_data=(X_val, y_val_onehot))

# Evaluate the model on the training set
y_train_pred = model.predict(X_train)
y_train_pred_classes = y_train_pred.argmax(axis=1)
y_train_classes = y_train_onehot.argmax(axis=1)
train_acc = accuracy_score(y_train_classes, y_train_pred_classes)
train_f1_micro = f1_score(y_train_classes, y_train_pred_classes, average='micro')
train_precision_micro = precision_score(y_train_classes, y_train_pred_classes, average='micro')
train_recall_micro = recall_score(y_train_classes, y_train_pred_classes, average='micro')

# Evaluate the model on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = y_test_pred.argmax(axis=1)
y_test_classes = y_test_onehot.argmax(axis=1)
test_acc = accuracy_score(y_test_classes, y_test_pred_classes)
test_f1_micro = f1_score(y_test_classes, y_test_pred_classes, average='micro')
test_precision_micro = precision_score(y_test_classes, y_test_pred_classes, average='micro')
test_recall_micro = recall_score(y_test_classes, y_test_pred_classes, average='micro')

# Print the results
print('Training accuracy:', train_acc)
print('Training F1 score (micro):', train_f1_micro)
print('Training precision (micro):', train_precision_micro)
print('Training recall (micro):', train_recall_micro)
print('Test accuracy:', test_acc)
print('Test F1 score (micro):', test_f1_micro)
print('Test precision (micro):', test_precision_micro)
print('Test recall (micro):', test_recall_micro)

Epoch 1/10
8083/8083 [==============================] - 17s 2ms/step - loss: 0.9683 - accuracy: 0.5134 - val_loss: 0.9343 - val_accuracy: 0.5356
Epoch 2/10
8083/8083 [==============================] - 16s 2ms/step - loss: 0.9027 - accuracy: 0.5559 - val_loss: 0.8824 - val_accuracy: 0.5658
Epoch 3/10
8083/8083 [==============================] - 16s 2ms/step - loss: 0.8588 - accuracy: 0.5830 - val_loss: 0.8490 - val_accuracy: 0.5847
Epoch 4/10
8083/8083 [==============================] - 16s 2ms/step - loss: 0.8303 - accuracy: 0.5982 - val_loss: 0.8220 - val_accuracy: 0.6019
Epoch 5/10
8083/8083 [==============================] - 16s 2ms/step - loss: 0.8105 - accuracy: 0.6091 - val_loss: 0.8059 - val_accuracy: 0.6125
Epoch 6/10
8083/8083 [==============================] - 16s 2ms/step - loss: 0.7953 - accuracy: 0.6174 - val_loss: 0.7948 - val_accuracy: 0.6168
Epoch 7/10
8083/8083 [==============================] - 16s 2ms/step - loss: 0.7823 - accuracy: 0.6238 - val_loss: 0.7907 - val_ac

In [48]:
class_labels = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=y_train)
class_weight_value = {class_labels[i]: weight for i, weight in enumerate(class_weights)}
class_weight_value = {i-1: w for i, w in class_weight_value.items()}
class_weight_value

{1: 0.3907614757863128, 2: 2.4805096165625384, 3: 26.489272632129776}

In [49]:
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(x_pca, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
input_dim = X_train.shape[1]
y_train_new = np.subtract(y_train, 1)
y_test_new = np.subtract(y_test, 1)
y_val_new = np.subtract(y_val, 1)
class_labels = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=y_train)
class_weight_value = {class_labels[i]: weight for i, weight in enumerate(class_weights)}
class_weight_value = {i-1: w for i, w in class_weight_value.items()}

# Convert the new class labels to a one-hot encoded array
y_train_onehot = to_categorical(y_train_new, num_classes=3)
y_test_onehot = to_categorical(y_test_new, num_classes=3)
y_val_onehot  =  to_categorical(y_val_new, num_classes=3)

# Define the model architecture
model = Sequential([
    Dense(64, activation='relu', input_shape=(input_dim,)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Train the model
model.fit(X_train, y_train_onehot, epochs=10, batch_size=32, validation_data=(X_val, y_val_onehot),class_weight=class_weight_value)

# Evaluate the model on the training set
y_train_pred = model.predict(X_train)
y_train_pred_classes = y_train_pred.argmax(axis=1)
y_train_classes = y_train_onehot.argmax(axis=1)
train_acc = accuracy_score(y_train_classes, y_train_pred_classes)
train_f1_micro = f1_score(y_train_classes, y_train_pred_classes, average='micro')
train_precision_micro = precision_score(y_train_classes, y_train_pred_classes, average='micro')
train_recall_micro = recall_score(y_train_classes, y_train_pred_classes, average='micro')

# Evaluate the model on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = y_test_pred.argmax(axis=1)
y_test_classes = y_test_onehot.argmax(axis=1)
test_acc = accuracy_score(y_test_classes, y_test_pred_classes)
test_f1_micro = f1_score(y_test_classes, y_test_pred_classes, average='micro')
test_precision_micro = precision_score(y_test_classes, y_test_pred_classes, average='micro')
test_recall_micro = recall_score(y_test_classes, y_test_pred_classes, average='micro')

# Print the results
print('Training accuracy:', train_acc)
print('Training F1 score (micro):', train_f1_micro)
print('Training precision (micro):', train_precision_micro)
print('Training recall (micro):', train_recall_micro)
print('Test accuracy:', test_acc)
print('Test F1 score (micro):', test_f1_micro)
print('Test precision (micro):', test_precision_micro)
print('Test recall (micro):', test_recall_micro)

Epoch 1/10
3164/3164 [==============================] - 7s 2ms/step - loss: 1.0250 - accuracy: 0.4756 - val_loss: 0.9883 - val_accuracy: 0.4924
Epoch 2/10
3164/3164 [==============================] - 6s 2ms/step - loss: 1.0001 - accuracy: 0.5029 - val_loss: 1.0359 - val_accuracy: 0.4720
Epoch 3/10
3164/3164 [==============================] - 7s 2ms/step - loss: 0.9919 - accuracy: 0.5115 - val_loss: 0.9884 - val_accuracy: 0.5314
Epoch 4/10
3164/3164 [==============================] - 7s 2ms/step - loss: 0.9866 - accuracy: 0.5232 - val_loss: 0.9563 - val_accuracy: 0.5506
Epoch 5/10
3164/3164 [==============================] - 7s 2ms/step - loss: 0.9802 - accuracy: 0.5229 - val_loss: 1.0241 - val_accuracy: 0.4920
Epoch 6/10
3164/3164 [==============================] - 7s 2ms/step - loss: 0.9743 - accuracy: 0.5291 - val_loss: 1.0417 - val_accuracy: 0.4967
Epoch 7/10
3164/3164 [==============================] - 7s 2ms/step - loss: 0.9690 - accuracy: 0.5274 - val_loss: 0.9937 - val_accuracy:

In [51]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(x_pca, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
input_dim = X_train.shape[1]
y_train_new = np.subtract(y_train, 1)
y_test_new = np.subtract(y_test, 1)
y_val_new = np.subtract(y_val, 1)
class_labels = np.unique(y_train)


# Convert the new class labels to a one-hot encoded array
y_train_onehot = to_categorical(y_train_new, num_classes=3)
y_test_onehot = to_categorical(y_test_new, num_classes=3)
y_val_onehot  =  to_categorical(y_val_new, num_classes=3)


# Define a list of model architectures to try
architectures = [
    [64, 32],
    [128, 64, 32],
    [256, 128, 64, 32],
]

# Loop through the model architectures and evaluate their performance
for i, architecture in enumerate(architectures):
    print('Model', i+1)
    print('Architecture:', architecture)

    # Define the model architecture
    model = Sequential()
    for j, layer_size in enumerate(architecture):
        if j == 0:
            model.add(Dense(layer_size, activation='relu', input_shape=(X_train.shape[1],)))
        else:
            model.add(Dense(layer_size, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train_onehot, epochs=10, batch_size=32, validation_data=(X_val, y_val_onehot))

    # Evaluate the model on the training set
    y_train_pred = model.predict(X_train)
    y_train_pred_classes = y_train_pred.argmax(axis=1)
    train_acc = accuracy_score(y_train, y_train_pred_classes)
    train_f1_micro = f1_score(y_train, y_train_pred_classes, average='micro')
    train_precision_micro = precision_score(y_train, y_train_pred_classes, average='micro')
    train_recall_micro = recall_score(y_train, y_train_pred_classes, average='micro')

    # Evaluate the model on the test set
    y_test_pred = model.predict(X_test)
    y_test_pred_classes = y_test_pred.argmax(axis=1)
    test_acc = accuracy_score(y_test, y_test_pred_classes)
    test_f1_micro = f1_score(y_test, y_test_pred_classes, average='micro')
    test_precision_micro = precision_score(y_test, y_test_pred_classes, average='micro')
    test_recall_micro = recall_score(y_test, y_test_pred_classes, average='micro')

    # Print the results
    print('Training accuracy:', train_acc)
    print('Training F1 score (micro):', train_f1_micro)
    print('Training precision (micro):', train_precision_micro)
    print('Training recall (micro):', train_recall_micro)
    print('Test accuracy:', test_acc)
    print('Test F1 score (micro):', test_f1_micro)
    print('Test precision (micro):', test_precision_micro)
    print('Test recall (micro):', test_recall_micro)
    print()

Model 1
Architecture: [64, 32]
Epoch 1/10
3164/3164 [==============================] - 7s 2ms/step - loss: 0.4520 - accuracy: 0.8510 - val_loss: 0.4495 - val_accuracy: 0.8488
Epoch 2/10
3164/3164 [==============================] - 6s 2ms/step - loss: 0.4411 - accuracy: 0.8530 - val_loss: 0.4468 - val_accuracy: 0.8488
Epoch 3/10
3164/3164 [==============================] - 6s 2ms/step - loss: 0.4386 - accuracy: 0.8530 - val_loss: 0.4445 - val_accuracy: 0.8488
Epoch 4/10
3164/3164 [==============================] - 6s 2ms/step - loss: 0.4370 - accuracy: 0.8530 - val_loss: 0.4450 - val_accuracy: 0.8488
Epoch 5/10
3164/3164 [==============================] - 6s 2ms/step - loss: 0.4357 - accuracy: 0.8530 - val_loss: 0.4462 - val_accuracy: 0.8488
Epoch 6/10
3164/3164 [==============================] - 6s 2ms/step - loss: 0.4348 - accuracy: 0.8530 - val_loss: 0.4451 - val_accuracy: 0.8488
Epoch 7/10
3164/3164 [==============================] - 6s 2ms/step - loss: 0.4338 - accuracy: 0.8530 - v

In [72]:
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(x_pca, y, test_size=0.2, random_state=42,stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42,stratify=y_train)
input_dim = X_train.shape[1]
y_train_new = np.subtract(y_train, 1)
y_test_new = np.subtract(y_test, 1)
y_val_new = np.subtract(y_val, 1)


# Convert the new class labels to a one-hot encoded array
y_train_onehot = to_categorical(y_train_new, num_classes=3)
y_test_onehot = to_categorical(y_test_new, num_classes=3)
y_val_onehot  =  to_categorical(y_val_new, num_classes=3)

# Define the model architecture
model = Sequential([
    Dense(256, activation='relu', input_shape=(input_dim,)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Train the model
model.fit(X_train, y_train_onehot, epochs=100, batch_size=32, validation_data=(X_val, y_val_onehot))

# Evaluate the model on the training set
y_train_pred = model.predict(X_train)
y_train_pred_classes = y_train_pred.argmax(axis=1)
y_train_classes = y_train_onehot.argmax(axis=1)
train_acc = accuracy_score(y_train_classes, y_train_pred_classes)
train_f1_micro = f1_score(y_train_classes, y_train_pred_classes, average='micro')
train_precision_micro = precision_score(y_train_classes, y_train_pred_classes, average='micro')
train_recall_micro = recall_score(y_train_classes, y_train_pred_classes, average='micro')

# Evaluate the model on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = y_test_pred.argmax(axis=1)
y_test_classes = y_test_onehot.argmax(axis=1)
test_acc = accuracy_score(y_test_classes, y_test_pred_classes)
test_f1_micro = f1_score(y_test_classes, y_test_pred_classes, average='micro')
test_precision_micro = precision_score(y_test_classes, y_test_pred_classes, average='micro')
test_recall_micro = recall_score(y_test_classes, y_test_pred_classes, average='micro')

# Print the results
print('Training accuracy:', train_acc)
print('Training F1 score (micro):', train_f1_micro)
print('Training precision (micro):', train_precision_micro)
print('Training recall (micro):', train_recall_micro)
print('Test accuracy:', test_acc)
print('Test F1 score (micro):', test_f1_micro)
print('Test precision (micro):', test_precision_micro)
print('Test recall (micro):', test_recall_micro)

Epoch 1/100
3164/3164 [==============================] - 69s 3ms/step - loss: 0.4566 - accuracy: 0.8513 - val_loss: 0.4732 - val_accuracy: 0.8516
Epoch 2/100
3164/3164 [==============================] - 8s 3ms/step - loss: 0.4505 - accuracy: 0.8516 - val_loss: 0.4483 - val_accuracy: 0.8516
Epoch 3/100
3164/3164 [==============================] - 8s 3ms/step - loss: 0.4516 - accuracy: 0.8516 - val_loss: 0.4968 - val_accuracy: 0.8516
Epoch 4/100
3164/3164 [==============================] - 8s 3ms/step - loss: 0.4504 - accuracy: 0.8516 - val_loss: 0.4504 - val_accuracy: 0.8516
Epoch 5/100
3164/3164 [==============================] - 8s 3ms/step - loss: 0.4517 - accuracy: 0.8516 - val_loss: 0.4490 - val_accuracy: 0.8516
Epoch 6/100
2614/3164 [=======================>......] - ETA: 1s - loss: 0.4519 - accuracy: 0.8509

KeyboardInterrupt: 

In [76]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Define a function that creates a Keras model
def create_model(learning_rate, num_hidden_layers):
    model = Sequential()
    model.add(Dense(32, input_shape=(input_dim,), activation='relu'))
    for i in range(num_hidden_layers):
        model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define the hyperparameters to search over
learning_rates = [0.001, 0.01, 0.1]
num_hidden_layers = [1, 2, 3]

# Create a KerasClassifier object
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32)

# Define the grid search parameters
param_grid = dict(learning_rate=learning_rates, num_hidden_layers=num_hidden_layers)

# Create a GridSearchCV object
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3,verbose=3)

# Fit the GridSearchCV object to the training data
grid_result = grid.fit(X_train, y_train)

# Print the results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
Epoch 1/10


C:\Users\express\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
C:\Users\express\.conda\envs\tensorflow\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


2110/2110 [==============================] - 4s 2ms/step - loss: 0.4559 - accuracy: 0.8507
Epoch 2/10
2110/2110 [==============================] - 3s 2ms/step - loss: 0.4438 - accuracy: 0.8518
Epoch 3/10
2110/2110 [==============================] - 3s 2ms/step - loss: 0.4406 - accuracy: 0.8519
Epoch 4/10
2110/2110 [==============================] - 3s 2ms/step - loss: 0.4387 - accuracy: 0.8519
Epoch 5/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4373 - accuracy: 0.8518
Epoch 6/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4361 - accuracy: 0.8518
Epoch 7/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4348 - accuracy: 0.8519
Epoch 8/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4340 - accuracy: 0.8518
Epoch 9/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4329 - accuracy: 0.8519
Epoch 10/10
1055/1055 [==============================] - 2s 1ms/step - loss: 0.4478 - accurac

C:\Users\express\.conda\envs\tensorflow\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


2110/2110 [==============================] - 4s 2ms/step - loss: 0.4585 - accuracy: 0.8488
Epoch 2/10
2110/2110 [==============================] - 3s 2ms/step - loss: 0.4442 - accuracy: 0.8514
Epoch 3/10
2110/2110 [==============================] - 3s 2ms/step - loss: 0.4413 - accuracy: 0.8514
Epoch 4/10
2110/2110 [==============================] - 3s 2ms/step - loss: 0.4397 - accuracy: 0.8514
Epoch 5/10
2110/2110 [==============================] - 3s 2ms/step - loss: 0.4378 - accuracy: 0.8514
Epoch 6/10
2110/2110 [==============================] - 3s 2ms/step - loss: 0.4370 - accuracy: 0.8514
Epoch 7/10
2110/2110 [==============================] - 3s 2ms/step - loss: 0.4360 - accuracy: 0.8513
Epoch 8/10
2110/2110 [==============================] - 3s 2ms/step - loss: 0.4352 - accuracy: 0.8514
Epoch 9/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4344 - accuracy: 0.8514
Epoch 10/10
1055/1055 [==============================] - 2s 1ms/step - loss: 0.4435 - accurac

...

2110/2110 [==============================] - 5s 2ms/step - loss: 0.4790 - accuracy: 0.8511
Epoch 2/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4658 - accuracy: 0.8519
Epoch 3/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4660 - accuracy: 0.8519
Epoch 4/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4660 - accuracy: 0.8519
Epoch 5/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4658 - accuracy: 0.8519
Epoch 6/10
2110/2110 [==============================] - 5s 2ms/step - loss: 0.4658 - accuracy: 0.8519
Epoch 7/10
2110/2110 [==============================] - 5s 2ms/step - loss: 0.4658 - accuracy: 0.8519
Epoch 8/10
2110/2110 [==============================] - 5s 2ms/step - loss: 0.4658 - accuracy: 0.8519
Epoch 9/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4658 - accuracy: 0.8519
Epoch 10/10
1055/1055 [==============================] - 2s 2ms/step - loss: 0.4649 - accurac

C:\Users\express\.conda\envs\tensorflow\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


2110/2110 [==============================] - 5s 2ms/step - loss: 0.4746 - accuracy: 0.8509
Epoch 2/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4661 - accuracy: 0.8514
Epoch 3/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4663 - accuracy: 0.8514
Epoch 4/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4660 - accuracy: 0.8514
Epoch 5/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4662 - accuracy: 0.8514
Epoch 6/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4665 - accuracy: 0.8514
Epoch 7/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4664 - accuracy: 0.8514
Epoch 8/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4658 - accuracy: 0.8514
Epoch 9/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4663 - accuracy: 0.8514
Epoch 10/10
1055/1055 [==============================] - 2s 2ms/step - loss: 0.4723 - accurac

C:\Users\express\.conda\envs\tensorflow\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


2110/2110 [==============================] - 5s 2ms/step - loss: 0.4788 - accuracy: 0.8506
Epoch 2/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4676 - accuracy: 0.8515
Epoch 3/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4676 - accuracy: 0.8515
Epoch 4/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4683 - accuracy: 0.8515
Epoch 5/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4678 - accuracy: 0.8515
Epoch 6/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4679 - accuracy: 0.8515
Epoch 7/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4681 - accuracy: 0.8515
Epoch 8/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4674 - accuracy: 0.8515
Epoch 9/10
2110/2110 [==============================] - 4s 2ms/step - loss: 0.4676 - accuracy: 0.8515
Epoch 10/10
1055/1055 [==============================] - 2s 2ms/step - loss: 0.4657 - accurac

In [ ]:
## The search grid shows that different architectures and parameters are not changing anything in the perfomrance, unfortunately
## We will not preceding with this model as NNs needs alot of data engineering work to do and this is beyond our knowledge 